In [1]:
from aligned import FileSource
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
store = await FileSource.json_at("feature-store.json").feature_store()

In [6]:
data = await store.feature_view("documentation_snippet").all().to_polars()

/Users/matsmollestad/.virtualenvs/feature-store/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
data.collect()

source_file,version_tag,snippet,0,combined_snippet,snippet_embedding
str,str,str,str,str,list[f32]
"""README.md""","""beta""","""# Aligned Ali...","""# Aligned Ali...","""# Aligned Ali...","[-0.078278, -0.058865, ... -0.033936]"
"""README.md""","""beta""","""# Feature View...","""# Feature View...","""# Feature View...","[0.028838, 0.011171, ... -0.053366]"
"""README.md""","""beta""","""# Data sources...","""# Data sources...","""# Data sources...","[0.026111, -0.052859, ... 0.020952]"
"""README.md""","""beta""","""## Fast develo...","""## Fast develo...","""## Fast develo...","[-0.028942, -0.03096, ... 0.005561]"
"""README.md""","""beta""","""# Describe Mod...","""# Describe Mod...","""# Describe Mod...","[0.043094, -0.032567, ... 0.043062]"
"""README.md""","""beta""","""# Data Enriche...","""# Data Enriche...","""# Data Enriche...","[0.007067, -0.077952, ... 0.013554]"
"""README.md""","""beta""","""# Access Data ...","""# Access Data ...","""# Access Data ...","[-0.034212, -0.051393, ... 0.034236]"
"""README.md""","""beta""",""" Select all fe...",""" Select all fe...",""" Select all fe...","[0.107537, -0.025253, ... 0.034834]"
"""README.md""","""beta""","""## Centraliced...","""## Centraliced...","""## Centraliced...","[0.002432, 0.001283, ... 0.035507]"


In [4]:
from examples.sources import redis
await store.with_source(redis).feature_view("documentation_snippet").batch_write(
    store.feature_view("documentation_snippet").all()
)

/Users/matsmollestad/.virtualenvs/feature-store/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
client = redis.redis()

In [53]:
data = await store.feature_view("documentation_snippet").process_input({
    "source_file": [""],
    "snippet": ["How can I compute features?"]
}).to_polars()
numpy_embedding = data.collect()["snippet_embedding"].to_numpy()

In [54]:
from redis.commands.search.query import Query

documents = await (client.ft("doc_index").search(
    Query("(@source_file:README.md)=>[KNN 5 @snippet_embedding $QUESTION as score]")
        .sort_by("score")
        .return_fields('score','source_file','combined_snippet')
        .dialect(2),
    {"QUESTION": numpy_embedding[0].tobytes()}
    )
)

In [56]:
for doc in documents.docs:
    print("\n\n", doc)



 Document {'id': 'feature_view:documentation_snippet:1', 'payload': None, 'score': '0.611296415329', 'source_file': 'README.md', 'combined_snippet': '# Feature Views\n\nWrite features as the should be, as data models.\nThen get code completion and typesafety by referencing them in other features.\n\nThis makes the features light weight, data source indipendent, and flexible.\n\n```python\nclass TitanicPassenger(FeatureView):\n\n    metadata = FeatureView.metadata_with(\n        name="passenger",\n        description="Some features from the titanic dataset",\n        batch_source=FileSource.csv_at("titanic.csv"),\n        stream_source=HttpStreamSource(topic_name="titanic")\n    )\n\n    passenger_id = Int32().as_entity()\n\n    # Input values\n    age = (\n        Float()\n            .description("A float as some have decimals")\n            .is_required()\n            .lower_bound(0)\n            .upper_bound(110)\n    )\n\n    name = String()\n    sex = String().accepted_values(["

In [59]:
print("In Aligned, features are computed within FeatureView classes. You define your features using methods and operators available in the framework. Here are a few examples to demonstrate how to compute features:\n\n1. Compute arithmetic features:\n\n```python\nclass ArithmeticFeatures(FeatureView):\n    metadata = FeatureView.metadata_with(\n        name=\"arithmetic_features\",\n        description=\"Demonstration of arithmetic features\",\n        batch_source=my_db.table(\"sample_table\"),\n    )\n\n    id = Int32().as_entity()\n\n    value1 = Float()\n    value2 = Float()\n\n    sum = value1 + value2\n    difference = value1 - value2\n    product = value1 * value2\n    quotient = value1 / value2\n```\n\n2. Compute date features:\n\n```python\nclass DateFeatures(FeatureView):\n    metadata = FeatureView.metadata_with(\n        name=\"date_features\",\n        description=\"Demonstration of date features\",\n        batch_source=my_db.table(\"sample_table\"),\n    )\n\n    id = Int32().as_entity()\n\n    event_timestamp = EventTimestamp()\n\n    day_of_week = event_timestamp.date_component(\"day\").description(\"The day of the week\")\n    month = event_timestamp.date_component(\"month\").description(\"The month\")\n    year = event_timestamp.date_component(\"year\").description(\"The year\")\n```\n\n3. Compute categorical features:\n\n```python\nclass CategoricalFeatures(FeatureView):\n    metadata = FeatureView.metadata_with(\n        name=\"categorical_features\",\n        description=\"Demonstration of categorical features\",\n        batch_source=my_db.table(\"sample_table\"),\n    )\n\n    id = Int32().as_entity()\n\n    color = String().accepted_values([\"red\", \"blue\", \"green\"])\n\n    is_red, is_blue, is_green = color.one_hot_encode([\"red\", \"blue\", \"green\"])\n```\n\n4. Compute geographical features:\n\n```python\nclass GeographicalFeatures(FeatureView):\n    metadata = FeatureView.metadata_with(\n        name=\"geographical_features\",\n        description=\"Demonstration of geographical features\",\n        batch_source=my_db.table(\"sample_table\"),\n    )\n\n    id = Int32().as_entity()\n\n    latitude1 = Float()\n    longitude1 = Float()\n    latitude2 = Float()\n    longitude2 = Float()\n\n    coordinate1 = Coordinate(latitude1, longitude1)\n    coordinate2 = Coordinate(latitude2, longitude2)\n\n    distance = coordinate1.eucledian_distance(coordinate2)\n```\n\nThese examples demonstrate how to compute various types of features within the Aligned framework. By defining features in the FeatureView classes, you get a clear data model, typesafety, and code completion. This makes it easy to combine and extend features as needed.")

In Aligned, features are computed within FeatureView classes. You define your features using methods and operators available in the framework. Here are a few examples to demonstrate how to compute features:

1. Compute arithmetic features:

```python
class ArithmeticFeatures(FeatureView):
    metadata = FeatureView.metadata_with(
        name="arithmetic_features",
        description="Demonstration of arithmetic features",
        batch_source=my_db.table("sample_table"),
    )

    id = Int32().as_entity()

    value1 = Float()
    value2 = Float()

    sum = value1 + value2
    difference = value1 - value2
    product = value1 * value2
    quotient = value1 / value2
```

2. Compute date features:

```python
class DateFeatures(FeatureView):
    metadata = FeatureView.metadata_with(
        name="date_features",
        description="Demonstration of date features",
        batch_source=my_db.table("sample_table"),
    )

    id = Int32().as_entity()

    event_timestamp = EventTimesta